In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install gym matplotlib numpy pandas scipy

In [7]:
import itertools
import numpy as np
import sys
import gym

# Since lib.envs isn't available, we'll need to define these environments here
# or use gym environments directly. For now, I'll create simplified versions.

from collections import defaultdict

# Simple CliffWalkingEnv implementation
class CliffWalkingEnv(gym.Env):
    def __init__(self):
        self.shape = (4, 12)
        self.start_state_index = np.ravel_multi_index((3, 0), self.shape)
        self.goal_state_index = np.ravel_multi_index((3, 11), self.shape)
        self.cliff = list(range(np.ravel_multi_index((3, 1), self.shape),
                               np.ravel_multi_index((3, 11), self.shape)))
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left

        # Calculate transition probabilities and rewards
        self.P = {}
        for s in range(self.nS):
            position = np.unravel_index(s, self.shape)
            self.P[s] = {a: [] for a in range(self.nA)}

            # Actions: 0=up, 1=right, 2=down, 3=left
            for a in range(self.nA):
                reward = -1.0  # default reward for each move
                next_position = list(position)
                if a == 0:
                    next_position[0] = max(position[0] - 1, 0)
                elif a == 1:
                    next_position[1] = min(position[1] + 1, self.shape[1] - 1)
                elif a == 2:
                    next_position[0] = min(position[0] + 1, self.shape[0] - 1)
                elif a == 3:
                    next_position[1] = max(position[1] - 1, 0)

                next_state = np.ravel_multi_index(next_position, self.shape)

                # Check if we're at the cliff
                if s in self.cliff:
                    next_state = self.start_state_index
                    reward = -100.0

                # Check if we're at the goal
                done = next_state == self.goal_state_index

                self.P[s][a] = [(1.0, next_state, reward, done)]

        self.observation_space = gym.spaces.Discrete(self.nS)
        self.action_space = gym.spaces.Discrete(self.nA)

        self.reset()

    def step(self, action):
        state, reward, done, _ = self._step(action)
        self.s = state
        return (state, reward, done, {})

    def _step(self, action):
        (probs, next_state, reward, done) = self.P[self.s][action][0]
        return (next_state, reward, done, {})

    def reset(self):
        self.s = self.start_state_index
        return self.s

# Simple WindyGridworldEnv implementation (not fully used in this code)
class WindyGridworldEnv(gym.Env):
    def __init__(self):
        self.shape = (7, 10)
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left
        self.wind = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
        self.reset()

    def step(self, action):
        # Not implemented as it's not used in the main code
        pass

    def reset(self):
        self.s = np.ravel_multi_index((3, 0), self.shape)
        return self.s

from scipy.optimize import minimize, rosen, rosen_der
from scipy.optimize import Bounds

bounds = Bounds([-0.1, -0.1], [0.1, 0.1])

env = CliffWalkingEnv()

def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

# Update these paths to your actual file locations
Q_space = np.load("/content/drive/MyDrive/DoubleReinforcementLearningMDP-master/Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("/content/drive/MyDrive/DoubleReinforcementLearningMDP-master/Q-table-cliff.npz")["xxx"]

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)

betabeta = 0.8
def sample_policy(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def behavior_policy(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def target_dense(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return prob2

def behav_dense(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return prob2

# FIXED: Matching the original notebook exactly
def sarsa2(env, policy, policy2, num_episodes, discount_factor=1.0, Q_space2=Q_space2, alpha=0.6, epsilon=0.03):
    """
    Expected SARSA implementation matching the original notebook
    """
    # Initialize Q as a copy of Q_space2 (not zeros)
    Q = np.copy(Q_space2)
    episode_episode = []

    for i_episode in range(num_episodes):
        if (i_episode + 1) % 200 == 0:
            sys.stdout.flush()

        state = env.reset()
        action = policy2(state)
        episode = []

        for t in itertools.count():
            # Take a step
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))

            # Pick the next action
            next_action = policy2(next_state)

            # TD Update - Expected SARSA without importance sampling
            td_target = reward + discount_factor * np.sum(Q[next_state,:]*target_dense(next_state))
            td_delta = td_target - Q[state, action]
            Q[state, action] += alpha * td_delta  # No importance sampling correction

            if done:
                break

            action = next_action
            state = next_state

        episode_episode.append(episode)

    # Return only Q and episode_episode (matching original)
    return Q, episode_episode

bounds = Bounds([-0.2, -0.2], [0.2, 0.2])
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))

depth = 1
def mc_prediction(env, policy, policy2, episode_episode, Q_=1.0, num_episodes=100, discount_factor=1.0):
    """
    Monte Carlo prediction for policy evaluation
    """
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    returns_count2 = defaultdict(float)

    predic_list = []
    predic_list2 = []
    predic_list3 = []
    predic_list22 = []
    predic_list4 = []
    predic_list5 = np.ones(num_episodes)
    auxiauxi = []
    epiepi = []
    weight_list = np.zeros([num_episodes, 1000])
    weight_list2 = np.zeros([num_episodes, 1002])
    weight_list3 = np.zeros([num_episodes, 1002])
    marginal_weight = np.zeros([num_episodes, 1000])
    marginal_weight_2 = np.zeros([num_episodes, 1000])
    auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2_2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list_2 = np.zeros([num_episodes, 1000])
    auxi_list2 = np.zeros([num_episodes, 1000])
    reward_list = np.zeros([num_episodes, 1000])
    state_list = np.zeros([num_episodes, 1000])
    action_list = np.zeros([num_episodes, 1000])

    count_list = np.zeros(1000)
    episolode_longe_list = []

    for i_episode in range(num_episodes):
        if i_episode % 200 == 0:
            sys.stdout.flush()

        episode = episode_episode[i_episode]

        W = 1.0
        W_list = []
        episolode_longe_list.append(len(episode))

        weight_list2[i_episode, 0] = 1.0
        for t in range(len(episode)):
            state, action, reward = episode[t]
            reward_list[i_episode, t] = reward
            state_list[i_episode, t] = state
            action_list[i_episode, t] = action

            W = W*target_dense(state)[action]/behav_dense(state)[action]*discount_factor
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            W_list.append(W)
            weight_list[i_episode, t] = W_list[t]
            weight_list2[i_episode, t+1] = W_list[t]
            weight_list3[i_episode, t] = target_dense(state)[action]/behav_dense(state)[action]

            count_list[t] += 1.0

            if t==0:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
            else:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-W_list[t-1]*np.sum(probprob*Q_[state,:])

            if t==0:
                auxi_list2[i_episode, t] = W_list[t]-1.0
            else:
                auxi_list2[i_episode, t] = W_list[t]-W_list[t-1]

    print(np.max(np.array(episolode_longe_list)))

    weight_list_mean = np.mean(weight_list, 1)
    reward_list_mean = np.mean(reward_list, 1)
    auxi_list_mean = np.mean(auxi_list, 1)
    auxi_list2_mean = np.mean(auxi_list2, 1)

    val = []

    ##### IPW - Standard Importance Sampling
    for i in range(num_episodes):
        predic_list.append(np.sum(weight_list[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list))

    #### Marginalized-IPW

    for i in range(num_episodes):
        for j in range(episolode_longe_list[i]):
            marginal_weight[i,j] = np.mean(weight_list[:,j][(state_list[:,j]==state_list[i,j]) & (action_list[:,j]==action_list[i,j])])
            if j==0:
                marginal_weight_2[i,j] = weight_list3[i,j]
            else:
                marginal_weight_2[i,j] = np.mean(weight_list[:,j-1][(state_list[:,j]==state_list[i,j])])*weight_list3[i,j]


    for i_episode in range(num_episodes):
        for t in range(episolode_longe_list[i_episode]):
            state = int(state_list[i_episode,t])  # Using int instead of np.int for Python 3
            action = int(action_list[i_episode,t])  # Using int instead of np.int for Python 3
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            if t==0:
                marginal_auxi_list[i_episode,t] = marginal_weight[i_episode,t]*Q_[state,action]-np.sum(probprob*Q_[state,:])
                marginal_auxi_list_2[i_episode,t] = marginal_weight_2[i_episode,t]*Q_[state,action]-np.sum(probprob*Q_[state,:])
                auxi_list[i_episode,t] = weight_list[i_episode,t]*Q_[state,action]-np.sum(probprob*Q_[state,:])
            else:
                marginal_auxi_list[i_episode,t] = marginal_weight[i_episode,t]*(Q_[state,action])-marginal_weight[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
                marginal_auxi_list_2[i_episode,t] = marginal_weight_2[i_episode,t]*(Q_[state,action])-marginal_weight_2[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
                auxi_list[i_episode,t] = weight_list[i_episode,t]*(Q_[state,action])-weight_list[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))

            if t==0:
                marginal_auxi_list2[i_episode,t] = marginal_weight[i_episode,t]-1.0
                marginal_auxi_list2_2[i_episode,t] = marginal_weight_2[i_episode,t]-1.0
                auxi_list2[i_episode,t] = weight_list[i_episode,t]-1.0
            else:
                marginal_auxi_list2[i_episode,t] =  marginal_weight[i_episode,t]- marginal_weight[i_episode,t-1]
                marginal_auxi_list2_2[i_episode,t] =  marginal_weight_2[i_episode,t]- marginal_weight_2[i_episode,t-1]
                auxi_list2[i_episode,t] = weight_list[i_episode,t]-weight_list[i_episode,t-1]


    for i in range(num_episodes):
        predic_list2.append(np.sum(marginal_weight[i,:]*reward_list[i,:]))

    ### marginal ipw2  #### Using action and state
    val.append(np.mean(predic_list2))


    ### marginal ipw3#### Using only state
    for i in range(num_episodes):
        predic_list22.append(np.sum(marginal_weight_2[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list22))


    #### DR
    val.append(np.mean(predic_list)-np.mean(np.sum(auxi_list,1)))

    #### marginal DR 1  #### Using action and state
    val.append(np.mean(predic_list2)-np.mean(np.sum(marginal_auxi_list,1)))
    #### marginal DR 2   #### Using only state
    val.append(np.mean(predic_list22)-np.mean(np.sum(marginal_auxi_list_2,1)))

    return val

# Main experiment run - with sample splitting like the original
is_list = []
is2_list = []
is3_list = []
wis_list = []
wis2_list = []
dm_list = []
dr_list = []
dr2_list = []
dr3_list = []
bdr_list = []
drs_list = []
drs2_list = []
drss_list = []
mdr_list = []
mdr_list2 = []

sample_size = 1000
# In Python 3, integer division requires // instead of /
sample_size = sample_size // 2

for kkk in range(100):
    print(kkk)
    #### Sample splitting
    ### First fold
    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_1 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    ### Second fold
    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_2 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    V_10k = 0.5*(np.array(V_10k_1)+np.array(V_10k_2))
    is_list.append(np.mean(V_10k[0]))
    is2_list.append(np.mean(V_10k[1]))
    is3_list.append(np.mean(V_10k[2]))
    dr_list.append(np.mean(V_10k[3]))
    dr2_list.append(np.mean(V_10k[4]))
    dr3_list.append(np.mean(V_10k[5]))
    probprob = 0.9*Q_space[36,:] + 0.1*prob1
    dm_list.append(np.sum(probprob*predicted_Q[36,:]))

    # Save results periodically
    if (kkk + 1) % 10 == 0:
        np.savez(f"2estimator_list_ipw_{betabeta}_{sample_size}", a=np.array(is_list))
        np.savez(f"2estimator_list_ipw2_{betabeta}_{sample_size}", a=np.array(is3_list))
        np.savez(f"2estimator_list_dm_{betabeta}_{sample_size}", a=np.array(dm_list))
        np.savez(f"2estimator_list_dr_{betabeta}_{sample_size}", a=np.array(dr_list))
        np.savez(f"2estimator_list_dr2_{betabeta}_{sample_size}", a=np.array(dr3_list))

# Analysis of results
true = -42.49
def mse(aaa):
    """Calculate the Mean Squared Error correctly for comparison"""
    aaa = np.array(aaa)
    # Filter extreme values
    aaa = aaa[aaa > -100]
    # Original MSE calculation
    return [np.mean((((aaa-true)*(aaa-true)))), np.sqrt(np.var((aaa-true)*(aaa-true)))]

print("IPW:")
print(f"Mean: {np.mean(is_list)}")
print(f"MSE: {mse(is_list)}")

print("WIS:")
print(f"Mean: {np.mean(is3_list)}")  # Note: Original used is3_list for WIS
print(f"MSE: {mse(is3_list)}")

print("DM:")
print(f"Mean: {np.mean(dm_list)}")
print(f"MSE: {mse(dm_list)}")

print("DR:")
print(f"Mean: {np.mean(dr_list)}")
print(f"MSE: {mse(dr_list)}")

print("DR3:")
print(f"Mean: {np.mean(dr3_list)}")
print(f"MSE: {mse(dr3_list)}")

0
282
307
1
215
249
2
215
287
3
313
221
4
183
282
5
201
237
6
196
250
7
220
164
8
307
289
9
269
444
10
219
226
11
216
271
12
246
235
13
214
220
14
207
216
15
224
252
16
241
237
17
260
246
18
246
194
19
335
265
20
259
212
21
225
321
22
215
222
23
190
233
24
241
340
25
217
212
26
274
223
27
235
231
28
201
224
29
172
215
30
195
247
31
209
299
32
221
215
33
270
219
34
245
232
35
231
224
36
243
212
37
243
273
38
271
202
39
309
184
40
230
247
41
265
172
42
227
252
43
190
207
44
253
316
45
214
283
46
263
195
47
236
208
48
301
329
49
200
266
50
267
264
51
297
216
52
273
206
53
314
247
54
241
227
55
192
276
56
323
392
57
174
204
58
257
182
59
275
200
60
213
191
61
220
235
62
241
244
63
261
674
64
257
258
65
231
258
66
254
264
67
298
176
68
233
197
69
209
192
70
338
188
71
304
202
72
239
182
73
284
205
74
186
318
75
265
194
76
172
312
77
221
296
78
197
230
79
252
183
80
254
260
81
261
206
82
329
241
83
227
183
84
230
282
85
180
235
86
182
402
87
162
240
88
288
194
89
214
194
90
335
198
91
277
25

In [ ]:
import itertools
import numpy as np
import sys
import gym

# Since lib.envs isn't available, we'll need to define these environments here
# or use gym environments directly. For now, I'll create simplified versions.

from collections import defaultdict

# Simple CliffWalkingEnv implementation
class CliffWalkingEnv(gym.Env):
    def __init__(self):
        self.shape = (4, 12)
        self.start_state_index = np.ravel_multi_index((3, 0), self.shape)
        self.goal_state_index = np.ravel_multi_index((3, 11), self.shape)
        self.cliff = list(range(np.ravel_multi_index((3, 1), self.shape),
                               np.ravel_multi_index((3, 11), self.shape)))
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left

        # Calculate transition probabilities and rewards
        self.P = {}
        for s in range(self.nS):
            position = np.unravel_index(s, self.shape)
            self.P[s] = {a: [] for a in range(self.nA)}

            # Actions: 0=up, 1=right, 2=down, 3=left
            for a in range(self.nA):
                reward = -1.0  # default reward for each move
                next_position = list(position)
                if a == 0:
                    next_position[0] = max(position[0] - 1, 0)
                elif a == 1:
                    next_position[1] = min(position[1] + 1, self.shape[1] - 1)
                elif a == 2:
                    next_position[0] = min(position[0] + 1, self.shape[0] - 1)
                elif a == 3:
                    next_position[1] = max(position[1] - 1, 0)

                next_state = np.ravel_multi_index(next_position, self.shape)

                # Check if we're at the cliff
                if s in self.cliff:
                    next_state = self.start_state_index
                    reward = -100.0

                # Check if we're at the goal
                done = next_state == self.goal_state_index

                self.P[s][a] = [(1.0, next_state, reward, done)]

        self.observation_space = gym.spaces.Discrete(self.nS)
        self.action_space = gym.spaces.Discrete(self.nA)

        self.reset()

    def step(self, action):
        state, reward, done, _ = self._step(action)
        self.s = state
        return (state, reward, done, {})

    def _step(self, action):
        (probs, next_state, reward, done) = self.P[self.s][action][0]
        return (next_state, reward, done, {})

    def reset(self):
        self.s = self.start_state_index
        return self.s

# Simple WindyGridworldEnv implementation (not fully used in this code)
class WindyGridworldEnv(gym.Env):
    def __init__(self):
        self.shape = (7, 10)
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left
        self.wind = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
        self.reset()

    def step(self, action):
        # Not implemented as it's not used in the main code
        pass

    def reset(self):
        self.s = np.ravel_multi_index((3, 0), self.shape)
        return self.s

from scipy.optimize import minimize, rosen, rosen_der
from scipy.optimize import Bounds

bounds = Bounds([-0.1, -0.1], [0.1, 0.1])

env = CliffWalkingEnv()

def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

Q_space = np.load("/content/drive/MyDrive/Work/Estimators/DoubleReinforcement/Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("/content/drive/MyDrive/Work/Estimators/DoubleReinforcement/Q-table-real-cliff.npz")["xxx"] #Q-table-cliff.npz

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)

betabeta = 0.8
def sample_policy(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def behavior_policy(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def target_dense(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return prob2

def behav_dense(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return prob2

def sarsa2(env, policy, policy2, num_episodes, discount_factor=1.0, Q_space2=Q_space2, alpha=0.6, epsilon=0.03):

    Q = np.copy(Q_space2)
    episode_episode = []

    for i_episode in range(num_episodes):

        if (i_episode + 1) % 200 == 0:
            sys.stdout.flush()

        state = env.reset()
        action = policy2(state)

        episode = []

        for t in itertools.count():
            # Take a step
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            # Pick the next action
            next_action = policy2(next_state)

            # TD Update
            td_target = reward + discount_factor * np.sum(Q[next_state,:]*target_dense(next_state))
            td_delta = td_target - Q[state, action]
            Q[state, action] += alpha * td_delta

            if done:
                break

            action = next_action
            state = next_state

        episode_episode.append(episode)

    return Q, episode_episode

bounds = Bounds([-0.2, -0.2], [0.2, 0.2])
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))


depth = 1
def mc_prediction(env, policy, policy2, episode_episode, Q_=1.0, num_episodes=100, discount_factor=1.0):

    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    returns_count2 = defaultdict(float)

    predic_list = []
    predic_list2 = []
    predic_list3 = []
    predic_list22 = []
    predic_list4 = []
    predic_list5 = np.ones(num_episodes)
    auxiauxi = []
    epiepi = []
    weight_list = np.zeros([num_episodes, 1000])  # For bounded IPW
    weight_list2 = np.zeros([num_episodes, 1002])  # For bounded IPW
    weight_list3 = np.zeros([num_episodes, 1002])  # For bounded IPW
    marginal_weight = np.zeros([num_episodes, 1000])  # For bounded IPW
    marginal_weight_2 = np.zeros([num_episodes, 1000])  # For bounded IPW
    auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2_2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list_2 = np.zeros([num_episodes, 1000])
    auxi_list2 = np.zeros([num_episodes, 1000])
    reward_list = np.zeros([num_episodes, 1000])
    state_list = np.zeros([num_episodes, 1000])
    action_list = np.zeros([num_episodes, 1000])

    count_list = np.zeros(1000)
    episolode_longe_list = []


    for i_episode in range(num_episodes):

        if i_episode % 200 == 0:
            sys.stdout.flush()

        episode = episode_episode[i_episode]

        W = 1.0
        W_list = []
        episolode_longe_list.append(len(episode))

        weight_list2[i_episode, 0] = 1.0
        for t in range(len(episode)):
            state, action, reward = episode[t]
            reward_list[i_episode, t] = reward
            state_list[i_episode, t] = state
            action_list[i_episode, t] = action

            W = W*target_dense(state)[action]/behav_dense(state)[action]*discount_factor
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            W_list.append(W)
            weight_list[i_episode, t] = W_list[t]
            weight_list2[i_episode, t+1] = W_list[t]
            weight_list3[i_episode, t] = target_dense(state)[action]/behav_dense(state)[action]

            count_list[t] += 1.0

            if t==0:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
            else:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-W_list[t-1]*np.sum(probprob*Q_[state,:])

            if t==0:
                auxi_list2[i_episode, t] = W_list[t]-1.0
            else:
                auxi_list2[i_episode, t] = W_list[t]-W_list[t-1]

    print(np.max(np.array(episolode_longe_list)))


    weight_list_mean = np.mean(weight_list, 1)
    reward_list_mean = np.mean(reward_list, 1)
    auxi_list_mean = np.mean(auxi_list, 1)
    auxi_list2_mean = np.mean(auxi_list2, 1)

    val = []

    ##### IPW
    for i in range(num_episodes):
        predic_list.append(np.sum(weight_list[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list))

    #### Marginalized-IPW

    for i in range(num_episodes):
        for j in range(episolode_longe_list[i]):
            marginal_weight[i,j] = np.mean(weight_list[:,j][(state_list[:,j]==state_list[i,j]) & (action_list[:,j]==action_list[i,j])])
            if j==0:
                marginal_weight_2[i,j] = weight_list3[i,j]
            else:
                marginal_weight_2[i,j] = np.mean(weight_list[:,j-1][(state_list[:,j]==state_list[i,j])])*weight_list3[i,j]


    for i_episode in range(num_episodes):
        for t in range(episolode_longe_list[i_episode]):
            state = int(state_list[i_episode, t])  # Changed np.int to int
            action = int(action_list[i_episode, t])  # Changed np.int to int
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            if t==0:
                marginal_auxi_list[i_episode, t] = marginal_weight[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
                marginal_auxi_list_2[i_episode, t] = marginal_weight_2[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
                auxi_list[i_episode, t] = weight_list[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
            else:
                marginal_auxi_list[i_episode, t] = marginal_weight[i_episode, t]*(Q_[state, action])-marginal_weight[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))
                marginal_auxi_list_2[i_episode, t] = marginal_weight_2[i_episode, t]*(Q_[state, action])-marginal_weight_2[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))
                auxi_list[i_episode, t] = weight_list[i_episode, t]*(Q_[state, action])-weight_list[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))

            if t==0:
                marginal_auxi_list2[i_episode, t] = marginal_weight[i_episode, t]-1.0
                marginal_auxi_list2_2[i_episode, t] = marginal_weight_2[i_episode, t]-1.0
                auxi_list2[i_episode, t] = weight_list[i_episode, t]-1.0
            else:
                marginal_auxi_list2[i_episode, t] = marginal_weight[i_episode, t]- marginal_weight[i_episode, t-1]
                marginal_auxi_list2_2[i_episode, t] = marginal_weight_2[i_episode, t]- marginal_weight_2[i_episode, t-1]
                auxi_list2[i_episode, t] = weight_list[i_episode, t]-weight_list[i_episode, t-1]


    for i in range(num_episodes):
        predic_list2.append(np.sum(marginal_weight[i,:]*reward_list[i,:]))

    ### marginal ipw2  #### Using action and state
    val.append(np.mean(predic_list2))


    ### marginal ipw3#### Using only state
    for i in range(num_episodes):
        predic_list22.append(np.sum(marginal_weight_2[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list22))


    #### DR
    val.append(np.mean(predic_list)-np.mean(np.sum(auxi_list, 1)))

    #### marginal DR 1  #### Using action and state
    val.append(np.mean(predic_list2)-np.mean(np.sum(marginal_auxi_list, 1)))
    #### marginal DR 2   #### Using only state
    val.append(np.mean(predic_list22)-np.mean(np.sum(marginal_auxi_list_2, 1)))

    return val

# Main experiment run
is_list = []
is2_list = []
is3_list = []
wis_list = []
wis2_list = []
dm_list = []
dr_list = []
dr2_list = []
dr3_list = []
bdr_list = []
drs_list = []
drs2_list = []
drss_list = []
mdr_list = []
mdr_list2 = []

sample_size = 1000
sample_size = sample_size // 2  # Integer division in Python 3
for kkk in range(100):
    print(kkk)
    #### Sample splititng
    ### First fold

    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_1 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    ### Second fold
    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_2 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    V_10k = 0.5*(np.array(V_10k_1)+np.array(V_10k_2))
    is_list.append(np.mean(V_10k[0]))
    is2_list.append(np.mean(V_10k[1]))
    is3_list.append(np.mean(V_10k[2]))
    dr_list.append(np.mean(V_10k[3]))
    dr2_list.append(np.mean(V_10k[4]))
    dr3_list.append(np.mean(V_10k[5]))
    probprob = 0.9*Q_space[36,:] + 0.1*prob1
    dm_list.append(np.sum(probprob*predicted_Q[36,:]))
    np.savez("2estimator_list_ipw_"+str(betabeta)+"_"+str(sample_size), a=is_list)
    np.savez("2estimator_list_ipw2_"+str(betabeta)+"_"+str(sample_size), a=is3_list)
    np.savez("2estimator_list_dm_"+str(betabeta)+"_"+str(sample_size), a=dm_list)
    np.savez("2estimator_list_dr_"+str(betabeta)+"_"+str(sample_size), a=dr_list)
    np.savez("2estimator_list_dr2_"+str(betabeta)+"_"+str(sample_size), a=dr3_list)

# Analysis of results
true = -42.49

# FIX: Properly calculate MSE instead of using hardcoded values
def mse(aaa):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]  # Filter extreme values
    mean_val = np.mean(aaa)  # Calculate mean
    bias = mean_val - true  # Calculate bias
    bias_squared = bias * bias  # Square the bias
    variance = np.var(aaa)  # Calculate variance
    mse_value = bias_squared + variance  # MSE = bias² + variance
    return [mse_value, np.sqrt(np.var((aaa-true)*(aaa-true)))]  # Return MSE and RMSE

print(np.mean(is_list))
print(mse(is_list))
print("wis")
print(np.mean(is3_list))
print(mse(is3_list))
print("dm")
print(np.mean(dm_list))
print(mse(dm_list))
print("dr")
print(np.mean(dr_list))
print(mse(dr_list))
print("dr3")
print(np.mean(dr3_list))
print(mse(dr3_list))

0
237
201
1
293
251
2
269
269
3
307
382
4
262
190
5
232
196
6
232
291
7
316
241
8
224
206
9
274
274
10
261
260
11
250
254
12
196
260
13
324
285
14
259
250
15
235
198
16
267
234
17
250
233
18
205
190
19
196
208
20
340
200
21
233
269
22
228
218
23
246
266
24
238
288
25
248
424
26
254
258
27
240
275
28
213
194
29
202
251
30
174
256
31
227
252
32
182
388
33
212
282
34
333
242
35
214
198
36
245
234
37
218
245
38
258
229
39
236
303
40
229
219
41
265
169
42
217
186
43
222
158
44
374
243
45
290
264
46
291
219
47
260
220
48
210
217
49
259
199
50
190
209
51
238
211
52
217
208
53
309
209
54
159
241
55
184
326
56
204
243
57
188
201
58
249
278
59
238
210
60
179
234
61
188
211
62
156
177
63
220
261
64
177
244
65
247
234
66
394
214
67
226
221
68
200
208
69
537
306
70
196
214
71
237
225
72
205
270
73
207
226
74
322
196
75
305
251
76
276
232
77
193
222
78
296
216
79
338
249
80
219
169
81
240
219
82
148
335
83
177
220
84
265
205
85
261
310
86
277
223
87
247
238
88
225
251
89
302
208
90
239
197
91
196
23

In [5]:
import itertools
import numpy as np
import sys
import gym

# Since lib.envs isn't available, we'll need to define these environments here
# or use gym environments directly. For now, I'll create simplified versions.

from collections import defaultdict

# Simple CliffWalkingEnv implementation
class CliffWalkingEnv(gym.Env):
    def __init__(self):
        self.shape = (4, 12)
        self.start_state_index = np.ravel_multi_index((3, 0), self.shape)
        self.goal_state_index = np.ravel_multi_index((3, 11), self.shape)
        self.cliff = list(range(np.ravel_multi_index((3, 1), self.shape),
                               np.ravel_multi_index((3, 11), self.shape)))
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left

        # Calculate transition probabilities and rewards
        self.P = {}
        for s in range(self.nS):
            position = np.unravel_index(s, self.shape)
            self.P[s] = {a: [] for a in range(self.nA)}

            # Actions: 0=up, 1=right, 2=down, 3=left
            for a in range(self.nA):
                reward = -1.0  # default reward for each move
                next_position = list(position)
                if a == 0:
                    next_position[0] = max(position[0] - 1, 0)
                elif a == 1:
                    next_position[1] = min(position[1] + 1, self.shape[1] - 1)
                elif a == 2:
                    next_position[0] = min(position[0] + 1, self.shape[0] - 1)
                elif a == 3:
                    next_position[1] = max(position[1] - 1, 0)

                next_state = np.ravel_multi_index(next_position, self.shape)

                # Check if we're at the cliff
                if s in self.cliff:
                    next_state = self.start_state_index
                    reward = -100.0

                # Check if we're at the goal
                done = next_state == self.goal_state_index

                self.P[s][a] = [(1.0, next_state, reward, done)]

        self.observation_space = gym.spaces.Discrete(self.nS)
        self.action_space = gym.spaces.Discrete(self.nA)

        self.reset()

    def step(self, action):
        state, reward, done, _ = self._step(action)
        self.s = state
        return (state, reward, done, {})

    def _step(self, action):
        (probs, next_state, reward, done) = self.P[self.s][action][0]
        return (next_state, reward, done, {})

    def reset(self):
        self.s = self.start_state_index
        return self.s

# Simple WindyGridworldEnv implementation (not fully used in this code)
class WindyGridworldEnv(gym.Env):
    def __init__(self):
        self.shape = (7, 10)
        self.nS = self.shape[0] * self.shape[1]
        self.nA = 4  # up, right, down, left
        self.wind = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
        self.reset()

    def step(self, action):
        # Not implemented as it's not used in the main code
        pass

    def reset(self):
        self.s = np.ravel_multi_index((3, 0), self.shape)
        return self.s

from scipy.optimize import minimize, rosen, rosen_der
from scipy.optimize import Bounds

bounds = Bounds([-0.1, -0.1], [0.1, 0.1])

env = CliffWalkingEnv()

def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

Q_space = np.load("/content/drive/MyDrive/Work/Estimators/DoubleReinforcement/Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("/content/drive/MyDrive/Work/Estimators/DoubleReinforcement/Q-table-cliff.npz")["xxx"] #Q-table-cliff.npz

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)

betabeta = 0.8
def sample_policy(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def behavior_policy(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return np.random.choice(env.nA, 1, p=prob2)[0]


def target_dense(observation, alpha=0.9):
    prob2 = alpha*Q_space[observation,:] + (1-alpha)*prob1
    return prob2

def behav_dense(observation, beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return prob2

def sarsa2(env, policy, policy2, num_episodes, discount_factor=1.0, Q_space2=Q_space2, alpha=0.6, epsilon=0.03):

    Q = np.copy(Q_space2)
    episode_episode = []

    for i_episode in range(num_episodes):

        if (i_episode + 1) % 200 == 0:
            sys.stdout.flush()

        state = env.reset()
        action = policy2(state)

        episode = []

        for t in itertools.count():
            # Take a step
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            # Pick the next action
            next_action = policy2(next_state)

            # TD Update
            td_target = reward + discount_factor * np.sum(Q[next_state,:]*target_dense(next_state))
            td_delta = td_target - Q[state, action]
            Q[state, action] += alpha * td_delta

            if done:
                break

            action = next_action
            state = next_state

        episode_episode.append(episode)

    return Q, episode_episode

bounds = Bounds([-0.2, -0.2], [0.2, 0.2])
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))


depth = 1
def mc_prediction(env, policy, policy2, episode_episode, Q_=1.0, num_episodes=100, discount_factor=1.0):

    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    returns_count2 = defaultdict(float)

    predic_list = []
    predic_list2 = []
    predic_list3 = []
    predic_list22 = []
    predic_list4 = []
    predic_list5 = np.ones(num_episodes)
    auxiauxi = []
    epiepi = []
    weight_list = np.zeros([num_episodes, 1000])  # For bounded IPW
    weight_list2 = np.zeros([num_episodes, 1002])  # For bounded IPW
    weight_list3 = np.zeros([num_episodes, 1002])  # For bounded IPW
    marginal_weight = np.zeros([num_episodes, 1000])  # For bounded IPW
    marginal_weight_2 = np.zeros([num_episodes, 1000])  # For bounded IPW
    auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list = np.zeros([num_episodes, 1000])
    marginal_auxi_list2_2 = np.zeros([num_episodes, 1000])
    marginal_auxi_list_2 = np.zeros([num_episodes, 1000])
    auxi_list2 = np.zeros([num_episodes, 1000])
    reward_list = np.zeros([num_episodes, 1000])
    state_list = np.zeros([num_episodes, 1000])
    action_list = np.zeros([num_episodes, 1000])

    count_list = np.zeros(1000)
    episolode_longe_list = []


    for i_episode in range(num_episodes):

        if i_episode % 200 == 0:
            sys.stdout.flush()

        episode = episode_episode[i_episode]

        W = 1.0
        W_list = []
        episolode_longe_list.append(len(episode))

        weight_list2[i_episode, 0] = 1.0
        for t in range(len(episode)):
            state, action, reward = episode[t]
            reward_list[i_episode, t] = reward
            state_list[i_episode, t] = state
            action_list[i_episode, t] = action

            W = W*target_dense(state)[action]/behav_dense(state)[action]*discount_factor
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            W_list.append(W)
            weight_list[i_episode, t] = W_list[t]
            weight_list2[i_episode, t+1] = W_list[t]
            weight_list3[i_episode, t] = target_dense(state)[action]/behav_dense(state)[action]

            count_list[t] += 1.0

            if t==0:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
            else:
                auxi_list[i_episode, t] = W_list[t]*Q_[state, action]-W_list[t-1]*np.sum(probprob*Q_[state,:])

            if t==0:
                auxi_list2[i_episode, t] = W_list[t]-1.0
            else:
                auxi_list2[i_episode, t] = W_list[t]-W_list[t-1]

    print(np.max(np.array(episolode_longe_list)))


    weight_list_mean = np.mean(weight_list, 1)
    reward_list_mean = np.mean(reward_list, 1)
    auxi_list_mean = np.mean(auxi_list, 1)
    auxi_list2_mean = np.mean(auxi_list2, 1)

    val = []

    ##### IPW
    for i in range(num_episodes):
        predic_list.append(np.sum(weight_list[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list))

    #### Marginalized-IPW

    for i in range(num_episodes):
        for j in range(episolode_longe_list[i]):
            marginal_weight[i,j] = np.mean(weight_list[:,j][(state_list[:,j]==state_list[i,j]) & (action_list[:,j]==action_list[i,j])])
            if j==0:
                marginal_weight_2[i,j] = weight_list3[i,j]
            else:
                marginal_weight_2[i,j] = np.mean(weight_list[:,j-1][(state_list[:,j]==state_list[i,j])])*weight_list3[i,j]


    for i_episode in range(num_episodes):
        for t in range(episolode_longe_list[i_episode]):
            state = int(state_list[i_episode, t])  # Changed np.int to int
            action = int(action_list[i_episode, t])  # Changed np.int to int
            probprob = 0.9*Q_space[state,:] + 0.1*prob1
            if t==0:
                marginal_auxi_list[i_episode, t] = marginal_weight[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
                marginal_auxi_list_2[i_episode, t] = marginal_weight_2[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
                auxi_list[i_episode, t] = weight_list[i_episode, t]*Q_[state, action]-np.sum(probprob*Q_[state,:])
            else:
                marginal_auxi_list[i_episode, t] = marginal_weight[i_episode, t]*(Q_[state, action])-marginal_weight[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))
                marginal_auxi_list_2[i_episode, t] = marginal_weight_2[i_episode, t]*(Q_[state, action])-marginal_weight_2[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))
                auxi_list[i_episode, t] = weight_list[i_episode, t]*(Q_[state, action])-weight_list[i_episode, t-1]*np.sum(probprob*(Q_[state,:]))

            if t==0:
                marginal_auxi_list2[i_episode, t] = marginal_weight[i_episode, t]-1.0
                marginal_auxi_list2_2[i_episode, t] = marginal_weight_2[i_episode, t]-1.0
                auxi_list2[i_episode, t] = weight_list[i_episode, t]-1.0
            else:
                marginal_auxi_list2[i_episode, t] = marginal_weight[i_episode, t]- marginal_weight[i_episode, t-1]
                marginal_auxi_list2_2[i_episode, t] = marginal_weight_2[i_episode, t]- marginal_weight_2[i_episode, t-1]
                auxi_list2[i_episode, t] = weight_list[i_episode, t]-weight_list[i_episode, t-1]


    for i in range(num_episodes):
        predic_list2.append(np.sum(marginal_weight[i,:]*reward_list[i,:]))

    ### marginal ipw2  #### Using action and state
    val.append(np.mean(predic_list2))


    ### marginal ipw3#### Using only state
    for i in range(num_episodes):
        predic_list22.append(np.sum(marginal_weight_2[i,:]*reward_list[i,:]))

    val.append(np.mean(predic_list22))


    #### DR
    val.append(np.mean(predic_list)-np.mean(np.sum(auxi_list, 1)))

    #### marginal DR 1  #### Using action and state
    val.append(np.mean(predic_list2)-np.mean(np.sum(marginal_auxi_list, 1)))
    #### marginal DR 2   #### Using only state
    val.append(np.mean(predic_list22)-np.mean(np.sum(marginal_auxi_list_2, 1)))

    return val

# Main experiment run
is_list = []
is2_list = []
is3_list = []
wis_list = []
wis2_list = []
dm_list = []
dr_list = []
dr2_list = []
dr3_list = []
bdr_list = []
drs_list = []
drs2_list = []
drss_list = []
mdr_list = []
mdr_list2 = []

sample_size = 1000
sample_size = sample_size // 2  # Integer division in Python 3
for kkk in range(100):
    print(kkk)
    #### Sample splititng
    ### First fold

    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_1 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    ### Second fold
    predicted_Q, episode_episode = sarsa2(env, sample_policy, behavior_policy, sample_size)
    V_10k_2 = mc_prediction(env, sample_policy, behavior_policy, episode_episode, predicted_Q, num_episodes=sample_size)

    V_10k = 0.5*(np.array(V_10k_1)+np.array(V_10k_2))
    is_list.append(np.mean(V_10k[0]))
    is2_list.append(np.mean(V_10k[1]))
    is3_list.append(np.mean(V_10k[2]))
    dr_list.append(np.mean(V_10k[3]))
    dr2_list.append(np.mean(V_10k[4]))
    dr3_list.append(np.mean(V_10k[5]))
    probprob = 0.9*Q_space[36,:] + 0.1*prob1
    dm_list.append(np.sum(probprob*predicted_Q[36,:]))
    np.savez("2estimator_list_ipw_"+str(betabeta)+"_"+str(sample_size), a=is_list)
    np.savez("2estimator_list_ipw2_"+str(betabeta)+"_"+str(sample_size), a=is3_list)
    np.savez("2estimator_list_dm_"+str(betabeta)+"_"+str(sample_size), a=dm_list)
    np.savez("2estimator_list_dr_"+str(betabeta)+"_"+str(sample_size), a=dr_list)
    np.savez("2estimator_list_dr2_"+str(betabeta)+"_"+str(sample_size), a=dr3_list)

# Analysis of results
true = -42.49

# FIX: Properly calculate MSE instead of using hardcoded values
def mse(aaa):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]  # Filter extreme values
    mean_val = np.mean(aaa)  # Calculate mean
    bias = mean_val - true  # Calculate bias
    bias_squared = bias * bias  # Square the bias
    variance = np.var(aaa)  # Calculate variance
    mse_value = bias_squared + variance  # MSE = bias² + variance
    return [mse_value, np.sqrt(np.var((aaa-true)*(aaa-true)))]  # Return MSE and RMSE

print(np.mean(is_list))
print(mse(is_list))
print("wis")
print(np.mean(is3_list))
print(mse(is3_list))
print("dm")
print(np.mean(dm_list))
print(mse(dm_list))
print("dr")
print(np.mean(dr_list))
print(mse(dr_list))
print("dr3")
print(np.mean(dr3_list))
print(mse(dr3_list))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Work/Estimators/DoubleReinforcement/Q-table-cliff.npz'